# We must incept!

* Makes an inception block as described in the " Going Deeper with Convolutions (2014)" piece by google


Firstly we must import keras

In [1]:
import tensorflow.keras as keras

In our case we are given some hints

* A_prev is the output from the previous layer
* filters is a tuple or list containing F1, F3R, F3,F5R, F5, FPP, respectively:
* F1 is the number of filters in the 1x1 convolution
* F3R is the number of filters in the 1x1 convolution before the 3x3 convolution
* F3 is the number of filters in the 3x3 convolution
* F5R is the number of filters in the 1x1 convolution before the 5x5 convolution
* F5 is the number of filters in the 5x5 convolution
* FPP is the number of filters in the 1x1 convolution after the max pooling
* All convolutions inside the inception block should use a rectified linear activation (ReLU)
* Returns: the concatenated output of the inception block

In [97]:
def inception_block(A_prev, filters):
    """
    Makes the Google inception block

    :param A_prev: input tensor from previous layer

    :param filters: filters is a tuple or list containing
    F1, F3R, F3,F5R, F5, FPP, respectively
 
    all conv layers must use ReLU activation function
    
    Returns: the concatenated output of the inception block
"""
    layers = keras.layers
    f1 = filters[0]
    f3r = filters[1]
    f3 = filters[2]
    f5r = filters[3]
    f5 = filters[4]
    fpp = filters[5]
    
    tower_1 = layers.Conv2D(
            filters=f1,
            kernel_size=(1, 1),
            padding='same',
            activation='relu',
        )
    
    tower_2 = layers.Conv2D(
            filters=f3r,
            kernel_size=(1, 1),
            padding='same',
            activation='relu',
        )
    
    tower_3 = layers.Conv2D(
        filters=f3,
        kernel_size=(3, 3),
        padding='same',
        activation='relu',
    )
    
    tower_4 = layers.Conv2D(
        filters=f5r,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )
    
    tower_5 = layers.Conv2D(
        filters=f5,
        kernel_size=(5, 5),
        padding='same',
        activation='relu',
    )
    
    pool = layers.MaxPooling2D((3, 3), strides=(1, 1),
                                  padding='same')(A_prev)
    
    tower_6 = layers.Conv2D(
        filters=fpp,
        kernel_size=(1, 1),
        padding='same',
        activation='relu',
    )
    
    layer_1 = tower_1(A_prev)
    layer_3_prev= tower_2(A_prev)
    layer_3 = tower_3(layer_3_prev)
    layer_5_prev= tower_4(A_prev)
    layer_5 = tower_5(layer_5_prev)
    layer_end = tower_6(pool)
    
    output = layers.concatenate([layer_1, layer_3, layer_5, layer_end])
    
    return output
    

#### Lets check out the output

In [98]:
#!/usr/bin/env python3

if __name__ == '__main__':
    X = keras.Input(shape=(224, 224, 3))
    Y = inception_block(X, [64, 96, 128, 16, 32, 32])
    model = keras.models.Model(inputs=X, outputs=Y)
    model.summary()

Model: "functional_69"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_46      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_262 (Conv2D) │ (None, 224, 224,  │        384 │ input_layer_46[0… │
│                     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_264 (Conv2D) │ (None, 224, 224,  │         64 │ input_layer_46[0… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_45    │ (None, 224, 224,  │          0 │ input_layer_46[0… │
│ (MaxPooling2D)      │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_261 (Conv2D) │ (None, 224, 224,  │        256 │ input_layer_46[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_263 (Conv2D) │ (None, 224, 224,  │    110,720 │ conv2d_262[0][0]  │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_265 (Conv2D) │ (None, 224, 224,  │     12,832 │ conv2d_264[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_266 (Conv2D) │ (None, 224, 224,  │        128 │ max_pooling2d_45… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_37      │ (None, 224, 224,  │          0 │ conv2d_261[0][0], │
│ (Concatenate)       │ 256)              │            │ conv2d_263[0][0], │
│                     │                   │            │ conv2d_265[0][0], │
│                     │                   │            │ conv2d_266[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,384 (485.88 KB)

 Trainable params: 124,384 (485.88 KB)

 Non-trainable params: 0 (0.00 B)